In [42]:
import sklearn
import os
import pandas as pd
from zipfile import ZipFile
from fuzzywuzzy import fuzz
from itertools import chain
from io import BytesIO
from urllib.request import urlopen
import ipywidgets as widgets
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import pandas as pd
path = "C:/Users/chloe/Polygence/Dataset"
# ratings_df = pd.read_csv(path + "/ratings.csv")
# movies_df = pd.read_csv(path + "/movies.csv")
# tags_df = pd.read_csv(path + "/tags.csv")
# links_df = pd.read_csv(path + "/links.csv")
# genome-scores_df = pd.read_csv(path + "/genome-scores.csv")


In [3]:
ratings = pd.read_csv(path + "/ratings.csv")
movies = pd.read_csv(path + "/movies.csv")
tags = pd.read_csv(path + "/tags.csv")
links = pd.read_csv(path + "/links.csv")
genome_scores = pd.read_csv(path + "/genome-scores.csv")

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [6]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [7]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    
    return results

In [10]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value = "Toy Story",
    description = "Movie Title:",
    disabled = False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 3:
            display(search(title))

movie_input.observe(on_type, names = 'value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [11]:
movie_id = 89745
movie = movies[movies["movieId"] == movie_id]

In [12]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [13]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [14]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [15]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [16]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [17]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())


In [18]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [19]:
rec_percentages

,similar,all
89745,1.000000,0.040459
58559,0.573393,0.148256
59315,0.530649,0.054931
79132,0.519715,0.132987
2571,0.496687,0.247010
...,...,...
47610,0.103545,0.022770
780,0.103380,0.054723
88744,0.103048,0.010383
1258,0.101226,0.083887


In [20]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]


In [21]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [22]:
rec_percentages

,similar,all,score
89745,1.000000,0.040459,24.716368
106072,0.103711,0.005289,19.610199
122892,0.241054,0.012367,19.491770
102125,0.216534,0.012119,17.867419
88140,0.215043,0.012052,17.843074
...,...,...,...
296,0.288933,0.288146,1.002730
593,0.222830,0.228665,0.974483
527,0.199967,0.217833,0.917984
1193,0.100895,0.120244,0.839081


In [23]:
rec_percentages.head(10).merge(movies, left_index=True, right_on = "movieId")


,similar,all,score,movieId,title,genres,clean_title
17067,1.000000,0.040459,24.716368,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
20513,0.103711,0.005289,19.610199,106072,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX,Thor The Dark World 2013
25058,0.241054,0.012367,19.491770,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,Avengers Age of Ultron 2015
19678,0.216534,0.012119,17.867419,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX,Iron Man 3 2013
16725,0.215043,0.012052,17.843074,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War,Captain America The First Avenger 2011
16312,0.175447,0.010142,17.299824,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,Thor 2011
21348,0.287608,0.016737,17.183667,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,Captain America The Winter Soldier 2014
25071,0.214049,0.012856,16.649399,122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller,Captain America Civil War 2016
25061,0.136017,0.008573,15.865628,122900,Ant-Man (2015),Action|Adventure|Sci-Fi,AntMan 2015
14628,0.242876,0.015517,15.651921,77561,Iron Man 2 (2010),Action|Adventure|Sci-Fi|Thriller|IMAX,Iron Man 2 2010


In [26]:
# Genre-based filtering additions

# Create distinct list of genres
unique_genres = sorted(list(set(list(chain(*[i.split('|') for i in movies.genres.unique().tolist()])))))
unique_genres.pop(0) # Get rid of "no genres listed" element in list
unique_genres.append("All Genres")

# create an object for TfidfVectorizer
tfidf_vector = TfidfVectorizer(stop_words='english')
# apply the object to the genres column
tfidf_matrix = tfidf_vector.fit_transform(unique_genres)

from sklearn.metrics.pairwise import linear_kernel
# create the cosine similarity matrix
sim_matrix = linear_kernel(tfidf_matrix,tfidf_matrix)

# Function to find most similar genre
def find_closest_genre(genre):
   leven_scores = [(element, fuzz.ratio(element, genre)) for element in unique_genres]
   sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True)
   closest_genre = sorted_leven_scores[0][0]
   distance_score = sorted_leven_scores[0][1]
   return closest_genre, distance_score   

In [27]:
# Year filtering additions

# Function to parse the year from a movie title
def find_year(title):
  year = title[title.find('(')+1:title.find(')')]
  if year.isnumeric():
    if (len(year) == 4):
      return year
    else:
      pass


In [28]:
movies["year"] = movies["title"].apply(find_year)
movies

,movieId,title,genres,clean_title,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995,1995
...,...,...,...,...,...
62418,209157,We (2018),Drama,We 2018,2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001,2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018,2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001,2001


In [29]:
# Create distinct list of years

unique_years = movies.year.copy().unique().tolist()
unique_years.remove(None)
unique_years.append("All Years")
unique_years

['1995',
 '1994',
 '1996',
 '1976',
 '1992',
 '1993',
 '1977',
 '1965',
 '1982',
 '1990',
 '1991',
 '1989',
 '1937',
 '1940',
 '1969',
 '1981',
 '1970',
 '1968',
 '1980',
 '1988',
 '1975',
 '1986',
 '1948',
 '1943',
 '1964',
 '1950',
 '1946',
 '1973',
 '1997',
 '1974',
 '1972',
 '1998',
 '1933',
 '1952',
 '1951',
 '1957',
 '1961',
 '1958',
 '1954',
 '1934',
 '1944',
 '1959',
 '1960',
 '1963',
 '1942',
 '1941',
 '1953',
 '1939',
 '1947',
 '1945',
 '1955',
 '1938',
 '1935',
 '1936',
 '1956',
 '1926',
 '1949',
 '1932',
 '1971',
 '1979',
 '1967',
 '1987',
 '1978',
 '1985',
 '1962',
 '1983',
 '1984',
 '1931',
 '1999',
 '1927',
 '1929',
 '1930',
 '1966',
 '1928',
 '1925',
 '1914',
 '2000',
 '1919',
 '1923',
 '1922',
 '1920',
 '1918',
 '1921',
 '2001',
 '2002',
 '2003',
 '1915',
 '1924',
 '2004',
 '1916',
 '1917',
 '2005',
 '2006',
 '1903',
 '2007',
 '2008',
 '2009',
 '2010',
 '1913',
 '2011',
 '1894',
 '1912',
 '2012',
 '1910',
 '1898',
 '2013',
 '1896',
 '2014',
 '2015',
 '1895',
 '1909',
 

In [30]:
def find_similar_movies(movie_id, genre_filter, year_filter):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .010]

    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 2)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)

    # ADDITION 1: Changed the filtering of the results to incorporate genre filters
    # Incorporate year filters
    unfiltered_df = rec_percentages.merge(movies, left_index=True, right_on = "movieId")[["score", "title", "genres", "year"]]

    if genre_filter == "All Genres":
      filtered_genre_df = unfiltered_df.head(10)
    else:
      filtered_genre_df = unfiltered_df[unfiltered_df.genres.str.contains(genre_filter)]
      filtered_genre_df.genres = filtered_genre_df.genres.str.replace('|',', ', regex=True)
    
    if year_filter == "All Years":
      filtered_year_df = filtered_genre_df.head(10)
      
    else:
      filtered_year_df = filtered_genre_df[filtered_genre_df.year.str.contains(year_filter, na = False)].head(10)
    
    output = filtered_year_df.reset_index().rename(columns={'index': 'movie_id'})

    return output

# find_similar_movies(89745, 'Action', '2012') # Test the function with The Avengers and an added genre filter

In [31]:
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display

w_genre = widgets.Dropdown(
    options = sorted(unique_genres), 
    description = 'Genre:',
    value = 'All Genres'
)

w_year = widgets.Dropdown(
    options = sorted(unique_years),
    description = 'Release Year:',
    value = 'All Years'
)

w_title = widgets.Text(
    value = 'Toy Story',
    description = 'Movie Title:',
    disabled = False
)

def movie_recommender(genre, year, title):
    if len(title) > 3:
        results = search(title) 
        movie_id = results.iloc[0]["movieId"]
        display(find_similar_movies(movie_id, genre, year))

w_box = interactive(movie_recommender, genre=w_genre, year=w_year, title=w_title)

display(w_box)

interactive(children=(Dropdown(description='Genre:', index=2, options=('Action', 'Adventure', 'All Genres', 'A…